In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
from minio import Minio
from minio.error import (ResponseError, BucketAlreadyOwnedByYou,
                         BucketAlreadyExists, BucketNotEmpty)
from k12libs.utils.nb_easy import k12ai_get_top_dir

In [ ]:
minio_server='s3-internal.didiyunapi.com'
minio_access_key='AKDD002E38WR1J7RMPTGRIGNVCVINY' # os.environ.get('MINIO_ACCESS_KEY'),
minio_secret_key='ASDDXYWs45ov7MNJbj5Wc2PM9gC0FSqCIkiyQkVC' # os.environ.get('MINIO_SECRET_KEY')

bucket_name = 'talentai'
pic = os.path.join(k12ai_get_top_dir(), 'assets/bear.jpg')

In [ ]:
mc = Minio(minio_server, 
           access_key=minio_access_key,
           secret_key=minio_secret_key,
           secure=True)

In [ ]:
buckets = mc.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

## remove_bucket

In [ ]:
try: 
    mc.remove_bucket(bucket_name)
except BucketNotEmpty as err:
    print(str(err))

## make_bucket

In [ ]:
try:
    mc.make_bucket(bucket_name)
except BucketAlreadyExists as err:
    print(str(err))

## list_buckets

In [ ]:
buckets = mc.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

## put_object

In [ ]:
with open(pic, 'rb') as file_data:
    file_stat = os.stat(pic)
    print('etag:', mc.put_object(bucket_name, 'subdir/assets/bear1.jpg',
                           file_data, file_stat.st_size))

## fput_object

In [ ]:
# larger_path = 'a'
# for i in range(512):
#     larger_path += 'a'
# print(larger_path)
# print(mc.fput_object(bucket_name, f'largerpath/{larger_path}/bear2.jpg', pic))

print(mc.fput_object(bucket_name, 'subdir/assets/bear2.jpg', pic))
print(mc.fput_object(bucket_name, 'subdir/assets/bear3.jpg', pic))

## fget_object

In [ ]:
print(mc.fget_object(bucket_name, 'subdir/assets/bear2.jpg', '/tmp/bear2.jpg'))
!ls -l /tmp/bear2.jpg

## list_objects

In [ ]:
for obj in mc.list_objects(bucket_name, prefix='subdir/assets', recursive=True):
    print(obj)
    
print('-'*100)
    
for obj in mc.list_objects_v2(bucket_name, prefix='subdir', recursive=False):
    print(obj)

## stat_object

In [ ]:
print(mc.stat_object(bucket_name, 'subdir/assets/bear1.jpg'))
print(mc.stat_object(bucket_name, 'subdir/assets/bear2.jpg'))
# print(mc.stat_object(bucket_name, 'subdir/assets'))
for obj in mc.list_objects(bucket_name, prefix='subdir/assets/bear1.jpg', recursive=True):
    print(obj)

## get_object

In [ ]:
data = mc.get_object(bucket_name, 'subdir/assets/bear1.jpg')
with open('/tmp/bear.jpg', 'wb') as file_data:
    for d in data.stream():
        file_data.write(d)
        
!ls -l /tmp/bear.jpg

## remove_object

In [ ]:
print(mc.stat_object(bucket_name, 'subdir/assets/bear1.jpg'))
mc.remove_object(bucket_name, 'subdir/assets/bear1.jpg')

In [ ]:
def delete_dir(name, path):
    for obj in mc.list_objects(name, prefix=path, recursive=True):
        mc.remove_object(obj.bucket_name, obj.object_name)
delete_dir(bucket_name, 'subdir')

for obj in mc.list_objects(bucket_name, prefix='subdir', recursive=True):
    print(obj)

## Wrap Tools

In [ ]:
import time
def k12ai_object_put(localpath, remotepath, prefix='framework/k12ai/'):
    with open(localpath, 'rb') as file_data:
        file_stat = os.stat(localpath)
        try:
            btime = time.time() 
            etag = mc.put_object(bucket_name, prefix + remotepath, file_data, file_stat.st_size)
            etime = time.time() 
            return etag, btime, etime, file_stat.st_size
        except ResponseError as err:
            print(str(err))
        
# k12ai_object_put('/data/pretrained/cv/vgg19_bn-c79401a0.pth', '123/001/ckpts/vgg19_bn-c79401a0.pth')

k12ai_object_put('/data/config.json', 'data/config.json')

In [ ]:
def k12ai_object_get(localpath, remotepath, prefix='framework/k12ai/'):
    try:
        return mc.fget_object(bucket_name, prefix + remotepath, localpath)
    except ResponseError as err:
        print(str(err))
        
# k12ai_object_get('/tmp/cv/vgg19_bn-c79401a0.pth', '123/001/ckpts/vgg19_bn-c79401a0.pth')
k12ai_object_put('/tmp/config.json', 'data/config.json')

In [ ]:
cnf = '111111.json'
mc.fput_object(bucket_name, f'framework/k12ai/data/{cnf}', f'/data/{cnf}')
# mc.fget_object(bucket_name, f'framework/k12ai/data/{cnf}', f'/tmp/{cnf}')

# print(k12ai_object_put(f'/data/{cnf}', f'data/{cnf}'))
# k12ai_object_get(f'/tmp/{cnf}', f'data/{cnf}')

data = mc.get_object(bucket_name, f'framework/k12ai/data/{cnf}')
with open(f'/tmp/{cnf}', 'wb') as file_data: 
 for d in data.stream():
     file_data.write(d)

In [ ]:
bucket_name

In [ ]:
def k12ai_object_rmdir(remotepath, prefix='framework/k12ai/'):
    for obj in mc.list_objects(bucket_name, prefix=prefix + remotepath, recursive=True):
        print(obj.object_name)
        mc.remove_object(obj.bucket_name, obj.object_name)
        
k12ai_object_rmdir('123/')

In [ ]:
import tarfile

def tar(fname):
    t = tarfile.open(fname + ".tar.gz", "w:gz")
    for root, dir, files in os.walk(fname):
        print(root, dir, files)
        for file in files:
            fullpath = os.path.join(root, file)
            t.add(fullpath)
    t.close()
    
tar('/data/users/26')

In [ ]:
from k12libs.k12ai.k12ai_utils import k12ai_object_put

# mc.make_bucket('k12ai')

# k12ai_object_put(mc, '/data/samples/datasets.tar.gz')
# k12ai_object_put(mc, '/data/samples')
k12ai_object_put(mc, '/data/samples', prefix_map=['/data/', 'framework/'])

In [ ]:
from k12libs.k12ai.k12ai_utils import k12ai_object_get

k12ai_object_get(mc, 'framework/samples/', prefix_map=['framework', '/data/tmp'])

In [ ]:
from k12libs.k12ai.k12ai_utils import k12ai_object_remove
k12ai_object_remove(mc, 'framework/samples')